In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# import SeaIceDrift helper class
from sea_ice_drift import SeaIceDrift

In [ ]:
# set names of files
f1 = 'S1B_EW_GRDM_1SDH_20161005T101835.tif'
f2 = 'S1A_EW_GRDM_1SDH_20161005T142446.tif'
# create SeaIceDrift object
sid = SeaIceDrift(f1, f2)

In [ ]:
# run Feature Tracking
uft, vft, lon1ft, lat1ft, lon2ft, lat2ft = sid.get_drift_FT()

In [ ]:
# plot identified and matched keypoints
plt.plot(lon1ft, lat1ft, '.')
plt.plot(lon2ft, lat2ft, '.')

In [ ]:
# plot ice drift vectors calculated from the keypoints
plt.quiver(lon1ft, lat1ft, uft, vft, color='r',
           angles='xy', scale_units='xy', scale=0.5)

In [ ]:
# create longitude and latitude grids at satellite projection
step = 10
lon1pm, lat1pm = sid.n1.get_geolocation_grids(stepSize=step)
plt.subplot(1,2,1);plt.imshow(lon1pm);plt.colorbar();plt.title('Longitude grid')
plt.subplot(1,2,2);plt.imshow(lat1pm);plt.colorbar();plt.title('Latitude grid')

In [ ]:
# run pattern matching
upm, vpm, apm, rpm, hpm, lon2pm, lat2pm = sid.get_drift_PM(
                                    lon1pm, lat1pm,
                                    lon1ft, lat1ft,
                                    lon2ft, lat2ft)

In [ ]:
# plot results of pattern matching

# find cells with correlation over 0.4
gpi = rpm > 0.4

# convert lon/lat into cartesian coordinates (only valid)
x1pm, y1pm = sid.n1.transform_points(lon1pm[gpi], lat1pm[gpi], DstToSrc=1)
x2pm, y2pm = sid.n1.transform_points(lon2pm[gpi], lat2pm[gpi], DstToSrc=1)

# compute ice drift speed in cartesian coordinates
dx_pm = x2pm - x1pm
dy_pm = y2pm - y1pm

# plot vectors of sea ice drift from Feature Tracking, color by MCC
plt.figure(figsize=(10,10))
plt.imshow(sid.n1[1], cmap='gray')
plt.quiver(x1pm, y1pm, dx_pm, dy_pm, rpm[gpi],
           angles='xy', scale_units='xy', scale=0.5, cmap='jet')
plt.colorbar(shrink=0.7)